In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from mlxtend.externals.estimator_checks import NotFittedError
from mlxtend.utils import assert_raises
from mlxtend.regressor import StackingRegressor, StackingCVRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, KFold
from scipy import sparse
from numpy.testing import assert_almost_equal
from nose.tools import raises
from sklearn.base import clone

/home/kota/miniconda3/envs/mlxtend/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [25]:
import numpy as np
from scipy import sparse
from mlxtend.externals.estimator_checks import NotFittedError
from mlxtend.regressor import StackingCVRegressor
from mlxtend.utils import assert_raises
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.base import clone


# Some test data
np.random.seed(1)
X1 = np.sort(5 * np.random.rand(40, 1), axis=0)
X2 = np.sort(5 * np.random.rand(40, 2), axis=0)
X3 = np.zeros((40, 3))
y = np.sin(X1).ravel()
y[::5] += 3 * (0.5 - np.random.rand(8))
y2 = np.zeros((40,))

In [26]:
import random

random.seed(8)
w = np.array([random.random() for _ in range(40)])


lr = LinearRegression()
svr_lin = SVR(kernel='linear')
ridge = Ridge(random_state=1)
svr_rbf = SVR(kernel='rbf')
stack = StackingCVRegressor(regressors=[svr_lin, lr, ridge],
                            meta_regressor=svr_rbf, 
                            cv=KFold(4, shuffle=True, random_state=7))
stack.fit(X1, y, sample_weight=np.ones(40)).predict(X1)
mse = 0.21
got = np.mean((stack.predict(X1) - y) ** 2)
print(got)


0.20166969294437087


In [23]:
lr = LinearRegression()
svr_lin = SVR(kernel='linear')
ridge = Ridge(random_state=1)
svr_rbf = SVR(kernel='rbf')
stack = StackingCVRegressor(regressors=[svr_lin, lr, ridge],
                            meta_regressor=svr_rbf, 
                            cv=KFold(5, shuffle=True, random_state=5))
pred1 = stack.fit(X1, y, sample_weight=np.ones(40)).predict(X1)

# lr = LinearRegression()
# svr_lin = SVR(kernel='linear')
# ridge = Ridge(random_state=1)
# svr_rbf = SVR(kernel='rbf')
# stack = StackingCVRegressor(regressors=[svr_lin, lr, ridge],
#                             meta_regressor=svr_rbf, 
#                             cv=KFold(5, shuffle=True, random_state=5))
pred2 = stack.fit(X1, y).predict(X1)

print(pred1)
print(pred2)

[ 0.36775818  0.41422082  0.43800099  0.46889041  0.59272485  0.61231241
  0.62836283  0.7367122   0.7522366   0.80566971  0.84050448  0.86365339
  0.87532447  0.9710891   0.97014473  0.96966604  0.952006    0.87445258
  0.82740436  0.82668374  0.82182401  0.81682575  0.40477061  0.37905676
  0.28624875 -0.25799577 -0.32626711 -0.33209999 -0.35638635 -0.35838507
 -0.47926837 -0.59599037 -0.75945827 -0.84141441 -0.91029104 -0.91237472
 -0.9124045  -0.92923798 -0.94573335 -0.94170385]
[ 0.36775818  0.41422082  0.43800099  0.46889041  0.59272485  0.61231241
  0.62836283  0.7367122   0.7522366   0.80566971  0.84050448  0.86365339
  0.87532447  0.9710891   0.97014473  0.96966604  0.952006    0.87445258
  0.82740436  0.82668374  0.82182401  0.81682575  0.40477061  0.37905676
  0.28624875 -0.25799577 -0.32626711 -0.33209999 -0.35638635 -0.35838507
 -0.47926837 -0.59599037 -0.75945827 -0.84141441 -0.91029104 -0.91237472
 -0.9124045  -0.92923798 -0.94573335 -0.94170385]


In [27]:
from sklearn.ensemble import RandomForestClassifier
r = RandomForestClassifier()
r.fit()

TypeError: fit() missing 2 required positional arguments: 'X' and 'y'

In [28]:
from sklearn.naive_bayes import GaussianNB
g = GaussianNB()
g.fit()

TypeError: fit() missing 2 required positional arguments: 'X' and 'y'

In [30]:
from sklearn.linear_model import LogisticRegression
l = LogisticRegression()
l.fit()

TypeError: fit() missing 2 required positional arguments: 'X' and 'y'

In [3]:
def test_multivariate():
    lr = LinearRegression()
    svr_lin = SVR(kernel='linear')
    ridge = Ridge(random_state=1)
    svr_rbf = SVR(kernel='rbf')
    stack = StackingCVRegressor(regressors=[svr_lin, lr, ridge],
                                meta_regressor=svr_rbf, 
                                cv=KFold(5, shuffle=True, random_state=3))
    stack.fit(X2, y).predict(X2)
    mse = 0.20
    got = np.mean((stack.predict(X2) - y) ** 2)
    assert round(got, 2) == mse, '%f != %f' % (round(got, 2), mse)


def test_internals():
    lr = LinearRegression()
    regressors = [lr, lr, lr, lr, lr]
    #cv = 10
    stack = StackingCVRegressor(regressors=[lr, lr, lr, lr, lr],
                                meta_regressor=lr,
                                cv=KFold(10, shuffle=True, random_state=4))
    stack.fit(X3, y2)
    assert stack.predict(X3).mean() == y2.mean()
    assert stack.meta_regr_.intercept_ == 0.0
    assert stack.meta_regr_.coef_[0] == 0.0
    assert stack.meta_regr_.coef_[1] == 0.0
    assert stack.meta_regr_.coef_[2] == 0.0
    assert len(stack.regr_) == len(regressors)


def test_gridsearch_numerate_regr():
    svr_lin = SVR(kernel='linear')
    ridge = Ridge(random_state=1)
    svr_rbf = SVR(kernel='rbf')
    stack = StackingCVRegressor(regressors=[svr_lin, ridge, ridge],
                                meta_regressor=svr_rbf, 
                                cv=KFold(3, shuffle=True, random_state=4))

    params = {'ridge-1__alpha': [0.01, 1.0],
              'ridge-2__alpha': [0.01, 1.0],
              'svr__C': [0.01, 1.0],
              'meta-svr__C': [0.01, 1.0],
              'use_features_in_secondary': [True, False]}

    grid = GridSearchCV(estimator=stack,
                        param_grid=params,
                        cv=KFold(5, shuffle=True, random_state=5),
                        refit=True,
                        verbose=0)
    grid = grid.fit(X1, y)
    got = round(grid.best_score_, 1)
    print(got)
    assert got >= 0.1 and got <= 0.2, '%f is wrong' % got
test_gridsearch_numerate_regr()

def test_get_params():
    lr = LinearRegression()
    svr_rbf = SVR(kernel='rbf')
    ridge = Ridge(random_state=1)
    stregr = StackingCVRegressor(regressors=[ridge, lr],
                                 meta_regressor=svr_rbf, cv=cv3)

    got = sorted(list({s.split('__')[0] for s in stregr.get_params().keys()}))
    expect = ['cv',
              'linearregression',
              'meta-svr',
              'meta_regressor',
              'refit',
              'regressors',
              'ridge',
              'shuffle',
              'store_train_meta_features',
              'use_features_in_secondary']
    assert got == expect, got


def test_regressor_gridsearch():
    lr = LinearRegression()
    svr_rbf = SVR(kernel='rbf')
    ridge = Ridge(random_state=1)
    stregr = StackingCVRegressor(regressors=[lr],
                                 meta_regressor=svr_rbf, cv=cv3)

    params = {'regressors': [[ridge, lr], [lr, ridge, lr]]}

    grid = GridSearchCV(estimator=stregr,
                        param_grid=params,
                        cv=cv5,
                        refit=True)
    grid.fit(X1, y)

    assert len(grid.best_params_['regressors']) == 3


def test_predict_meta_features():
    lr = LinearRegression()
    svr_rbf = SVR(kernel='rbf')
    ridge = Ridge(random_state=1)
    stregr = StackingCVRegressor(regressors=[lr, ridge],
                                 meta_regressor=svr_rbf, cv=cv3)
    X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3)
    stregr.fit(X_train, y_train)
    test_meta_features = stregr.predict(X_test)
    assert test_meta_features.shape[0] == X_test.shape[0]


def test_train_meta_features_():
    lr = LinearRegression()
    svr_rbf = SVR(kernel='rbf')
    ridge = Ridge(random_state=1)
    stregr = StackingCVRegressor(regressors=[lr, ridge],
                                 meta_regressor=svr_rbf, cv=cv3,
                                 store_train_meta_features=True)
    X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3)
    stregr.fit(X_train, y_train)
    train_meta_features = stregr.train_meta_features_
    assert train_meta_features.shape[0] == X_train.shape[0]


def test_not_fitted_predict():
    lr = LinearRegression()
    svr_rbf = SVR(kernel='rbf')
    ridge = Ridge(random_state=1)
    stregr = StackingCVRegressor(regressors=[lr, ridge],
                                 meta_regressor=svr_rbf, cv=cv3,
                                 store_train_meta_features=True)
    X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.3)

    expect = ("This StackingCVRegressor instance is not fitted yet. Call "
              "'fit' with appropriate arguments before using this method.")

    assert_raises(NotFittedError,
                  expect,
                  stregr.predict,
                  X_train)

    assert_raises(NotFittedError,
                  expect,
                  stregr.predict_meta_features,
                  X_train)


def test_clone():
    lr = LinearRegression()
    svr_rbf = SVR(kernel='rbf')
    ridge = Ridge(random_state=1)
    stregr = StackingCVRegressor(regressors=[lr, ridge],
                                 meta_regressor=svr_rbf, cv=cv3,
                                 store_train_meta_features=True)
    clone(stregr)


def test_sparse_matrix_inputs():
    lr = LinearRegression()
    svr_lin = SVR(kernel='linear')
    ridge = Ridge(random_state=1)
    svr_rbf = SVR(kernel='rbf')
    stack = StackingCVRegressor(regressors=[svr_lin, lr, ridge],
                                meta_regressor=svr_rbf, cv=cv3)

    # dense
    stack.fit(X1, y).predict(X1)
    mse = 0.20
    got = np.mean((stack.predict(X1) - y) ** 2)
    assert round(got, 2) == mse

    # sparse
    stack.fit(sparse.csr_matrix(X1), y)
    mse = 0.20
    got = np.mean((stack.predict(sparse.csr_matrix(X1)) - y) ** 2)
    assert round(got, 2) == mse


def test_sparse_matrix_inputs_with_features_in_secondary():
    lr = LinearRegression()
    svr_lin = SVR(kernel='linear')
    ridge = Ridge(random_state=1)
    svr_rbf = SVR(kernel='rbf')
    stack = StackingCVRegressor(regressors=[svr_lin, lr, ridge],
                                meta_regressor=svr_rbf, cv=cv3,
                                use_features_in_secondary=True)

    # dense
    stack.fit(X1, y).predict(X1)
    mse = 0.20
    got = np.mean((stack.predict(X1) - y) ** 2)
    assert round(got, 2) == mse

    # sparse
    stack.fit(sparse.csr_matrix(X1), y)
    mse = 0.20
    got = np.mean((stack.predict(sparse.csr_matrix(X1)) - y) ** 2)
    assert round(got, 2) == mse

0.7


AssertionError: 0.700000 is wrong

In [ ]:
np.random.seed(1)
X1 = np.sort(5 * np.random.rand(40, 1), axis=0)
X2 = np.sort(5 * np.random.rand(40, 2), axis=0)
y = np.sin(X1).ravel()
y[::5] += 3 * (0.5 - np.random.rand(8))
y2 = np.sin(X2)

#w = np.random.random(40)
import random
random.seed(1)
w = np.array([random.random() for _ in range(40)])

In [ ]:
lr = LinearRegression()
svr_lin = SVR(kernel='linear')
ridge = Ridge(random_state=1)
svr_rbf = SVR(kernel='rbf')
stregr = StackingCVRegressor(regressors=[svr_lin, lr, ridge],
                             meta_regressor=svr_rbf, cv=cv)
stregr.fit(X1, y, sample_weight=w).predict(X1)
mse = 0.21
got = np.mean((stregr.predict(X1) - y) ** 2)
print(got)

In [ ]:
lr = LinearRegression()
svr_lin = SVR(kernel='linear')
ridge = Ridge(random_state=1)
svr_rbf = SVR(kernel='rbf')
stregr = StackingCVRegressor(regressors=[svr_lin, lr, ridge],
                             meta_regressor=svr_rbf, cv=cv)
stregr.fit(X1, y, sample_weight=np.ones(40)).predict(X1)
mse = 0.21
got = np.mean((stregr.predict(X1) - y) ** 2)
print(got)

In [ ]:
lr = LinearRegression()
svr_lin = SVR(kernel='linear')
ridge = Ridge(random_state=1)
svr_rbf = SVR(kernel='rbf')
stregr = StackingRegressor(regressors=[svr_lin, lr, ridge],
                           meta_regressor=svr_rbf)
pred1 = stregr.fit(X1, y).predict(X1)


lr = LinearRegression()
svr_lin = SVR(kernel='linear')
ridge = Ridge(random_state=1)
svr_rbf = SVR(kernel='rbf')
stregr = StackingRegressor(regressors=[svr_lin, lr, ridge],
                           meta_regressor=svr_rbf)
pred2 = stregr.fit(X1, y, np.ones(40)).predict(X1)
print(pred1)
print(pred2)

In [ ]:
print(ridge.fit(X1, y).predict(X1))
print(ridge.fit(X1, y, 2*np.ones(40)).predict(X1))

In [ ]:
print(ridge.fit(X1, y, ).predict(X1))
print(ridge.fit(X1, y, np.ones(40)).predict(X1))

In [ ]:
lr = LinearRegression()
svr_lin = SVR(kernel='linear')
ridge = Ridge(random_state=1)
svr_rbf = SVR(kernel='rbf')

stregr = StackingRegressor(regressors=[svr_lin, lr, ridge],
                           meta_regressor=MLPRegressor())
stregr.fit(X1, y, w).predict(X1)
mse = 0.21
got = np.mean((stregr.predict(X1) - y) ** 2)
got

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
import random

np.random.seed(1)
cv = KFold(2, shuffle=True)
print(list(cv.split([1,2,3,4,5,6])))

np.random.seed(1)
random.random()
cv = KFold(2, shuffle=True)
print(list(cv.split([1,2,3,4,5,6])))

In [ ]:
lr = LinearRegression()
svr_lin = SVR(kernel='linear')
ridge = Ridge(random_state=1)
svr_rbf = SVR(kernel='rbf')
stack = StackingCVRegressor(regressors=[svr_lin, lr, ridge],
                            meta_regressor=svr_rbf, cv=cv)
stack.fit(X1, y).predict(X1)
mse = 0.21
got = np.mean((stack.predict(X1) - y) ** 2)
got